# Initialization (RUN ALL BELLOW)

In [ ]:
pip install fmpsdk

In [22]:
import pandas as pd
from urllib.request import urlopen
import certifi
import ssl
import json
import fmpsdk
from dateutil.relativedelta import relativedelta
from openpyxl.utils import get_column_letter
#from google.colab import files
from tqdm import tqdm

# Create a secure SSL context
context_init = ssl.create_default_context(cafile=certifi.where())


file_2025='https://raw.githubusercontent.com/alexandre-tcc/target-client_tracking/refs/heads/main/TCC%20Pre-qualified%20Targets%20202504421.csv'

pd.read_csv(file_2025)


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sales($MM),NaN,NaN,NaN,Inventory ($MM),NaN,NaN,NaN
1,Ticker,Name,S&P LT LC,GICS_SECTOR_NAME,GICS_INDUSTRY_NAME,CNTRY_OF_RISK,CUR_MKT_CAP,CURRENT_TRR_YTD,EV_TO_T12M_EBITDA,VOLATILITY_360D,FY2021,FY2022,FY2023,FY2024,FY2021,FY2022,FY2023,FY2024
2,SJM US Equity,JM SMUCKER CO/THE,BBB,Consumer Staples,Food Products,US,"12,391.15",6.9,21.8,24.48,"8,002.7","7,998.9","8,529.2","8,178.7",959.9,1089.3,1009.8,1038.9
3,DKS US Equity,DICK'S SPORTING GOODS INC,BBB,Consumer Discretionary,Specialty Retail,US,"14,816.42",-18.4,11.8,41.55,"9,584.0","12,293.4","12,368.2","12,984.4",1953.568,2297.609,2830.917,2848.797
4,LDOS US Equity,LEIDOS HOLDINGS INC,BBB,Industrials,Professional Services,US,"17,875.58",-2.9,11.3,27.20,"13,737.0","14,396.0","15,438.0","16,662.0",274,287,310,315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,AMZN US Equity,AMAZON.COM INC,AA,Consumer Discretionary,Broadline Retail,US,"1,831,800.15",-21.3,17.8,31.16,"469,822.0","513,983.0","574,785.0","637,959.0",32640,34405,33318,34214
338,BRK/A US Equity,BERKSHIRE HATHAWAY INC-CL A,AA,Financials,Financial Services,US,"1,118,779.77",14.4,3.6,17.37,"354,745.0","234,121.0","439,337.0","424,232.0",20954,25366,24159,24008
339,WMT US Equity,WALMART INC,AA,Consumer Staples,Consumer Staples Distribution,US,"747,330.71",3.5,20.0,23.41,"559,151.0","572,754.0","611,289.0","648,125.0",44949,56511,56576,54892
340,JNJ US Equity,JOHNSON & JOHNSON,AAA,Health Care,Pharmaceuticals,US,"379,478.72",9.8,12.8,17.96,"93,775.0","79,990.0","85,159.0","88,821.0",10387,10268,11181,12444


Data extraction functions:

In [ ]:
def get_data_url(url):
    response = urlopen(url, context=context_init)
    data = response.read().decode("utf-8")
    data=json.loads(data)
    return pd.DataFrame(data)

def get_org_chart(ticker):
    apikey='2723105eaade3c1ca8d66ca5c567b590'
    url = (f'https://financialmodelingprep.com/api/v3/key-executives/{ticker}?apikey={apikey}')
    df_=get_data_url(url)
    df_=df_[['title']+['name']]
    df_=df_.drop_duplicates(subset='name')
    #df_=df_.drop_duplicates(subset='title', inplace=True)
    return df_

def stock_price(ticker):
    try:
        stock_prices = fmpsdk.historical_price_full(apikey='2723105eaade3c1ca8d66ca5c567b590', symbol=ticker)
        closing_prices = [price['close'] for price in stock_prices]

        weekly_stock_variation = (closing_prices[0]-closing_prices[7])/closing_prices[7]

        if weekly_stock_variation >=0:
            return "+"+str(round(weekly_stock_variation*100,2))+"%"
        else:
            return str(round(weekly_stock_variation*100,2))+"%"

    except Exception as e:
        print(f"Unexpected error: {e}")
        return "FAIL"


def data_inventory(ticker,period):

    apikey='2723105eaade3c1ca8d66ca5c567b590'

    symbol: str = ticker

    try:
        data_BSS=pd.DataFrame(fmpsdk.balance_sheet_statement(apikey=apikey, symbol=symbol, period=period))
        #return '$'+str((data_BSS['inventory'].iloc[0])/1000000)+"M"
        return data_BSS['inventory'].iloc[0]
    except Exception as e:
        print(f"Unexpected error: {e}")
        return "FAIL"


def get_latest_earnings_transcript(ticker):
    apikey = '2723105eaade3c1ca8d66ca5c567b590'

    url = f'https://financialmodelingprep.com/api/v3/earning_call_transcript/{ticker}?apikey={apikey}'

    try:
        data = get_data_url(url)
        date_latest=pd.to_datetime(data["date"].iloc[0])
        date_next=date_latest + relativedelta(months=3)
        transcript=data['content'].iloc[0]
        return transcript, date_next
    except Exception as e:
        print(f"Unexpected error: {e}")
        return "FAIL"

def return_company_profile(tck):
    # Get company profile and industry from the ticker
    company_info = fmpsdk.company_profile(symbol=tck, apikey='2723105eaade3c1ca8d66ca5c567b590')
    try:
        return pd.DataFrame([company_info[0]])
    except Exception as e:
        print(f"Unexpected error: {e}")
        return pd.DataFrame([{'companyName': 'FAIL', 'industry': 'FAIL'}])

def new_executive(tck):
    return 'in dev.'

def credit_rating(tck):
    
    file_2023='https://raw.githubusercontent.com/alexandre-tcc/target-client_tracking/refs/heads/main/TCC%20companies%20mapping%2010142024.csv'
    file_2025='https://raw.githubusercontent.com/alexandre-tcc/target-client_tracking/refs/heads/main/TCC%20Pre-qualified%20Targets%20202504421.csv'
    
    df_2023=pd.read_csv(file_2023)
    df_2025=pd.read_csv(file_2025)
    
    df_rating_filtered_2025=df_2025[df_2025['ticker']==tck]

    if df_rating_filtered_2025.empty:
        df_rating_filtered_2023=df_2023[df_2023['ticker']==tck]
        if df_rating_filtered_2023.empty:
            return 'Need Manual research'
        else:
            return df_rating_filtered_2023['S&P rating'].iloc[0]
    else:
        return df_rating_filtered_2025['S&P rating'].iloc[0]

Output functions

In [ ]:
def generate_table(lst_tck):
    df_rtr = pd.DataFrame(columns=['Company Name', 'Ticker', 'Industry', 'Stock Trend (7days)', 'Inventory ($)', 'S&P Rating (2023)','Next Earnings', 'Latest Earnings Presentation','Exec. Update'])
    for tck in tqdm(lst_tck, desc="Progress", unit="ticker"):
        company_profil=return_company_profile(tck)
        data_earning_=get_latest_earnings_transcript(tck)

        stock_trend=stock_price(tck)
        next_call=data_earning_[1]
        transcript=data_earning_[0]
        inventory=data_inventory(tck, 'annual')
        company_name=company_profil['companyName'].iloc[0]
        company_industry=company_profil['industry'].iloc[0]
        credit_SnP=credit_rating(tck)
        exec_update=new_executive(tck)

        df_rtr.loc[len(df_rtr)]=[company_name, tck, company_industry, stock_trend, inventory, credit_SnP, next_call, transcript, exec_update]

    #df_rtr.to_excel('Target List Intelligence.xlsx', index=False)
    return df_rtr


def generate_output(list_tck):
    df=generate_table(list_tck)
    with pd.ExcelWriter('Target List Intelligence.xlsx', engine='openpyxl') as writer:
        df.to_excel(writer, index=False, sheet_name='Target List')
        worksheet = writer.sheets['Target List']

        for i in range(len(df.columns)):
            col_letter = get_column_letter(i + 1)  # 1-based indexing
            worksheet.column_dimensions[col_letter].width = 25  # or any width you want
    files.download('Target List Intelligence.xlsx')

# **Run Program**
## You can add/remove tickers from this list
## Generated file will automatically be downloaded

In [ ]:
tickers = ['MSFT', 'TSLA', 'AAPL', 'GOOG', 'AMZN', 'META', 'NVDA', 'BRK.B', 'JPM', 'UNH']

generate_output(tickers)

# Git COMMIT (DO NOT RUN)

In [19]:
!git add .


In [20]:
!git commit -m "revision"


[main 28560f4] revision
 1 file changed, 288 insertions(+), 191 deletions(-)


In [21]:
!git push origin main


To https://github.com/alexandre-tcc/target-client_tracking.git
   b171be9..28560f4  main -> main
